In [1]:
import os, sys
sys.path.append("..")
from config.load_config import load_yaml_config, to_dict, recursive_namespace
import torch
import numpy as np
from torch import Tensor
from importlib import reload

import pytorch_lightning as pl

___

### Load network and dataset parameters and generate dataset

In [20]:
from data.synthetic.SyntheticMeshPopulation import SyntheticMeshPopulation
from data.SyntheticDataModules import SyntheticMeshesDataset
from torch.utils.data import DataLoader
from utils.helpers import get_datamodule, get_coma_args

In [41]:
config = load_yaml_config("../config_files/config_folded_c_and_s.yaml")
dm = get_datamodule(config)
coma_args = get_coma_args(config, dm)

Retrieving synthetic population from cached file.


___

### `Decoder3DMesh`

In [23]:
import models.Model3D as Model3D
Model3D = reload(module=Model3D)
Decoder3DMesh = Model3D.Decoder3DMesh
Encoder3DMesh = Model3D.Encoder3DMesh

In [24]:
coma_args["num_conv_filters_dec"] = coma_args["num_conv_filters_dec_c"]
dec_config = {k: v for k,v in coma_args.items() if k in Model3D.DECODER_ARGS}
dec_config["latent_dim_content"] = 45

In [25]:
decoder_3d = Decoder3DMesh(**dec_config)

In [ ]:
z = Tensor(np.random.random(45)).unsqueeze(0)
s_hat = decoder_3d(z)

___

### `Encoder3DMesh`

In [ ]:
enc_config = {k: v for k,v in coma_args.items() if k in Model3D.ENCODER_ARGS}
enc_config["latent_dim_content"] = 45

In [26]:
datapoint = next(iter(dm.train_dataloader()))
s_t = datapoint["s_t"]
s = s_t[:,0,:,:] # take the first time point from the temporal sequence

In [28]:
encoder_3d = Encoder3DMesh(**enc_config)
mu, logvar = encoder_3d(s)

___

### `AutoEncoder3DMesh`

In [39]:
mu, logvar = encoder_3d(s)
s_hat = decoder_3d(mu)

torch.Size([16, 63, 32])
